In [1]:
%load_ext line_profiler

In [2]:
import cv2
import numpy as np
import knn
import inpainting_functions

import random

In [3]:
DEFINED = 0
UNDEFINED = 255

In [4]:
def create_pattern(mask: np.ndarray, origin: tuple, k: int) -> list[tuple]:
    origin_y, origin_x = origin
    neighbors = knn.nn_circular_native(mask, origin, k)
    neighbors_relative = []
    for y, x in neighbors:
        neighbors_relative.append((y - origin_y, x - origin_x))
    return neighbors_relative


In [5]:
def generate_candidates(mask: np.ndarray, neighbors_relative: list[tuple], n: int) -> list[tuple]:
    height, width = mask.shape
    candidates = []
    while len(candidates) < n:
        y, x = np.random.randint(0, height), np.random.randint(0, width)
        if mask[y, x] == UNDEFINED:
            continue
        for y_n, x_n in neighbors_relative:
            if y + y_n < 0 or y + y_n >= height or x + x_n < 0 or x + x_n >= width or mask[y + y_n, x + x_n] == UNDEFINED:
                break
        else:
            candidates.append((y, x))
    return candidates

In [6]:
def sparse_l2(image: np.ndarray, origin: tuple, candidate: tuple, neighbors: list[tuple]) -> float:
    origin_y, origin_x = origin
    candidate_y, candidate_x = candidate
    distance = 0
    for neighbor_y, neighbor_x in neighbors:
        distance += np.sum(np.power(image[origin_y + neighbor_y, origin_x + neighbor_x] - image[candidate_y + neighbor_y, candidate_x + neighbor_x], 2))

    return np.sqrt(distance)

In [7]:
def choose_candidate(image: np.ndarray, origin: tuple, candidates: list[tuple], neighbors: list[tuple]) -> tuple:
    distances = []
    for candidate in candidates:
        distances.append(sparse_l2(image, origin, candidate, neighbors))

    return candidates[np.argmin(distances)]

In [8]:
def choose_candidate_native_l2(image: np.ndarray, origin: tuple, candidates: list[tuple], neighbors: list[tuple]) -> tuple:
    distances = []
    for candidate in candidates:
        distances.append(inpainting_functions.sparse_l2(image, origin, candidate, neighbors))
    return candidates[np.argmin(distances)]

In [9]:
def inpainting(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

In [10]:
image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)

%lprun -f inpainting inpainting(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.979615 s
File: /tmp/ipykernel_13553/2782660982.py
Function: inpainting at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     185958.0 185958.0      0.0      image[mask == UNDEFINED] = UNDEFINED
     3         1    1452227.0    1e+06      0.1      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        260.0    260.0      0.0      iteration = 0
     5       101      26196.0    259.4      0.0      while undefined_pixels:
     6       101     315827.0   3127.0      0.0          origin = random.choice(undefined_pixels)
     7       101    6220276.0  61586.9      0.6          pattern = create_pattern(mask, origin, n_neighbors)
     8       101  231040709.0    2e+06     23.6          candidates = generate_candidates(mask, pattern, n_candidates

In [11]:
def inpainting_2(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)
%lprun -f inpainting_2 inpainting_2(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.267635 s
File: /tmp/ipykernel_13553/2995356549.py
Function: inpainting_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting_2(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     165549.0 165549.0      0.1      image[mask == UNDEFINED] = UNDEFINED
     3         1     451003.0 451003.0      0.2      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        181.0    181.0      0.0      iteration = 0
     5       101      22137.0    219.2      0.0      while undefined_pixels:
     6       101     241091.0   2387.0      0.1          origin = random.choice(undefined_pixels)
     7       101    5120022.0  50693.3      1.9          pattern = create_pattern(mask, origin, n_neighbors)
     8       101  230452776.0    2e+06     86.1          candidates = generate_candidates(mask, pattern, n_candid

In [12]:
def inpainting_3(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = inpainting_functions.generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)
%lprun -f inpainting_3 inpainting_3(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.0390832 s
File: /tmp/ipykernel_13553/1648953004.py
Function: inpainting_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting_3(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     203631.0 203631.0      0.5      image[mask == UNDEFINED] = UNDEFINED
     3         1     430586.0 430586.0      1.1      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        180.0    180.0      0.0      iteration = 0
     5       101      24321.0    240.8      0.1      while undefined_pixels:
     6       101     225457.0   2232.2      0.6          origin = random.choice(undefined_pixels)
     7       101    4731608.0  46847.6     12.1          pattern = create_pattern(mask, origin, n_neighbors)
     8       101    1191398.0  11796.0      3.0          candidates = inpainting_functions.generate_candidates(m

In [14]:
def inpainting_4(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 1000):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = inpainting_functions.generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image = cv2.imread("data/1d.jpg")
mask = cv2.imread("data/mask_1.png", cv2.IMREAD_GRAYSCALE)
inpainting_4(image, mask)